# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_fr import lexicon_fr, legend_fr, COLOURS_FR
from utils.io import boreholes_from_files, gdf_viewer
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/striplog/legend.py:687: UserWarning: This legend contains duplicate components.
  warnings.warn(w)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/striplog/legend.py:687: UserWarning: This legend contains duplicate components.
  warnings.warn(w)


## Reading of CSV files in a dir

In [2]:
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [3]:
df = pd.read_csv(save_dir+'Pilote_test.csv', sep=',')

In [4]:
gdf_viewer(df)

Rows : 86, columns : 10


interactive(children=(IntSlider(value=10, description='rows', max=86, min=10, readout=False), IntSlider(value=…

## Data Checking

In [5]:
df.drop(index=df.query('X!=X').index, inplace=True)
df.drop(index=df.query('Description!=Description').index, inplace=True)

In [6]:
df['Diam_for'] = df['Diam_for']/1000

In [7]:
df.query('ID=="201"')

,ID,Ep_litho,X,Y,Z,Long_for,Litho_base,Litho_top,Diam_for,Description
0,201,0.6,152890.245758,122596.47426,101.926886,2.4,1.2,0.6,0.075,Remblais avec briques de construction couleur ...
1,201,1.2,152890.245758,122596.47426,101.926886,2.4,2.4,1.2,0.075,Briquaille de déchets de construction et galet...


In [8]:
df['Ep_litho']=df['Litho_base']-df['Litho_top']

## Create Boreholes from dataframes or files

In [9]:
id_list = ['205','207','214']
df = df.query('ID in @id_list')
df.reset_index(drop=True, inplace=True)
bhs_df=[df] # list of dataframes

In [10]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Description',
                                                lexicon=lexicon_fr,
                                                diam_field='Diam_for', 
                                                litho_top_field='Litho_top',
                                                litho_base_field='Litho_base',
                                                thick_field='Ep_litho')


Dataframe 0 processing...
|__ID:'205'
Summary : [Striplog(7 Intervals, start=0.0, stop=4.8)]
{0: {'description': "Couche de boue en surface liée à l'intervention de Géosonda", 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f8e526755e0>, 'base': <core.orm.PositionOrm object at 0x7f8e52675a90>}, 1: {'description': 'Limon brun ', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f8e526753a0>, 'base': <core.orm.PositionOrm object at 0x7f8e526752b0>}, 2: {'description': 'Remblais gris gravillonneux avec présence de déchets de construction', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f8e526751c0>, 'base': <core.orm.PositionOrm object at 0x7f8e52675490>}, 3: {'description': "Présence d'eau avec remblais pollués (odeur forte de naphtalène avec irrisation de l'eau) + mélange de limon et gravillons ", 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f8e526754c0>, 'base': <core.orm.PositionOrm object at 0x7f8e52675460>}, 4: {'des

/home/yanathan/Projects/GSDMA/utils/io.py:478: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=bh_name))
/home/yanathan/Projects/GSDMA/utils/io.py:478: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Intervals.id to

In [11]:
litho_list = []
color_list = []
for i, c in components.items():
    if not hasattr(c, 'lithology'):
        c.lithology = 'sable'
        c.colour = 'rouge'
    elif c.lithology == 'sand':
        c.lithology = 'sable'
        c.colour = 'blanc'
    if c.lithology not in litho_list:
        litho_list.append(c.lithology)
    if hasattr(c, 'colour') and c.colour not in color_list:
        color_list.append(c.colour)

In [12]:
litho_list, color_list

(['remblais', 'limon', 'alluvions', 'boue', 'sable', 'limons'],
 ['verdâtre',
  'gris',
  'verdâtres',
  'brun',
  'rouge',
  'noire',
  'brique',
  'sable beige',
  'brun beige'])

In [13]:
components[9]

modifier,saturée
lithology,sable
colour,rouge


In [14]:
memoris_components = [Component({'lithology': 'remblais'}),
              Component({'lithology': 'limon'}),
              Component({'lithology': 'schistes'}),
              Component({'lithology': 'sable'}),
              Component({'lithology': 'anthropique'})
              ]
list_of_decors=[]
hatches = ['=', 'x', '.', 's', '=']
colours = ['#888888', '#882222', '#AAAAAA', '#CC22CC', '#CC2222']

for i in range(len(memoris_components)):
    if hasattr(memoris_components[i], 'colour'):
        c = memoris_components[i].colour
    else:
        c=colours[i]
    
    if hasattr(memoris_components[i], 'hatch'):
        h = memoris_components[i].hatch
    else:
        h=hatches[i]
        
    d = {'color': c,
         'hatch': h,
         'component': memoris_components[i],
         'width': 3}
    decor = Decor(d)
    list_of_decors.append(decor)
    
memoris_legend = Legend(list_of_decors)

In [15]:
len(boreholes), len(components)

(3, 20)

## Create a project and save data in a database

In [16]:
remove('tmp_files/tfe_orm_db.db')

In [17]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [18]:
Base.metadata.create_all(engine)

2021-06-18 16:02:31,281 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-18 16:02:31,282 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-06-18 16:02:31,283 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 16:02:31,284 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-06-18 16:02:31,285 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 16:02:31,285 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-06-18 16:02:31,286 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 16:02:31,286 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-06-18 16:02:31,287 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 16:02:31,288 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-06-18 16:02:31,289 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-18 16:02:31,289 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-06-18 16:02:31,289 INFO sqlalchemy.engine.Engine 

In [19]:
Session = sessionmaker(bind=engine)
session = Session()

In [20]:
p = Project(session, legend=legend_fr, lexicon=lexicon_fr)
p.add_components(components)

2021-06-18 16:02:31,335 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-18 16:02:31,339 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-18 16:02:31,339 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ()
2021-06-18 16:02:31,345 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-06-18 16:02:31,346 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ((0, 'Remblais, naphtalène'), (1, 'Limon, grossiers, verdâtre'), (2, 'Remblais, grossiers, naphtalène'), (3, 'Remblais, gris'), (4, 'Alluvions, sableux, verdâtres'), (5, 'Limon, brun'), (6, 'Boue'), (7, 'Saturée, rouge, sable')  ... displaying 10 of 20 total bound parameter sets ...  (18, 'Remblais, verdâtre'), (19, 'Remblais, brun beige'))
2021-06-18 16:02:31,347 INFO sqlalchemy.engine.Engine COMMIT
2021-06-18 16:02:31,350 INFO sqlalchemy.engine.Engine B

Add boreholes into the project

In [21]:
for bh in boreholes:
    p.add_borehole(bh)

2021-06-18 16:02:31,359 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-18 16:02:31,360 INFO sqlalchemy.engine.Engine [generated in 0.00133s] ('205', 4.8, 0.075)
2021-06-18 16:02:31,363 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-18 16:02:31,364 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ((0, 0.0, 0.0, 0.0, 152887.69317500002, 122594.619977), (1, 0.2, 0.2, 0.2, 152887.69317500002, 122594.619977), (2, 0.2, 0.2, 0.2, 152887.69317500002, 122594.619977), (3, 0.4, 0.4, 0.4, 152887.69317500002, 122594.619977), (4, 0.4, 0.4, 0.4, 152887.69317500002, 122594.619977), (5, 1.2, 1.2, 1.2, 152887.69317500002, 122594.619977), (6, 1.2, 1.2, 1.2, 152887.69317500002, 122594.619977), (7, 2.4, 2.4, 2.4, 152887.69317500002, 122594.619977)  ... displaying 10 of 14 total bound parameter sets ...  (12, 3.6, 3.6, 3.6, 152887.69317500002, 122594.619977), (13, 4.8, 4.8, 4

2021-06-18 16:02:31,418 INFO sqlalchemy.engine.Engine [cached since 0.03247s ago] (12,)
2021-06-18 16:02:31,420 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-18 16:02:31,421 INFO sqlalchemy.engine.Engine [cached since 0.03141s ago] (13,)
Borehole geometry created successfully !
Borehole geometry created successfully !
2021-06-18 16:02:31,432 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-18 16:02:31,433 INFO sqlalchemy.engine.Engine [cached since 0.07385s ago] ('207', 4.8, 0.075)
2021-06-18 16:02:31,434 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-18 16:02:31,435 INFO sqlalchemy.engine.Engine [cached

2021-06-18 16:02:31,474 INFO sqlalchemy.engine.Engine [cached since 0.08498s ago] (25,)
2021-06-18 16:02:31,476 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-18 16:02:31,477 INFO sqlalchemy.engine.Engine [cached since 0.09088s ago] (26,)
2021-06-18 16:02:31,478 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-18 16:02:31,479 INFO sqlalchemy.engine.Engine [cached since 0.08923s ago] (27,)
Borehole geometry created successfully !
Borehole geometry created successfully !
2021-0

2021-06-18 16:02:31,537 INFO sqlalchemy.engine.Engine [cached since 0.1509s ago] (38,)
2021-06-18 16:02:31,539 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-18 16:02:31,540 INFO sqlalchemy.engine.Engine [cached since 0.1506s ago] (39,)
2021-06-18 16:02:31,541 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-18 16:02:31,542 INFO sqlalchemy.engine.Engine [cached since 0.1559s ago] (40,)
2021-06-18 16:02:31,543 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_i

In [22]:
p.add_link_between_components_and_intervals(link_intv_comp)

2021-06-18 16:02:31,566 INFO sqlalchemy.engine.Engine INSERT INTO "Linkintervalcomponent" (int_id, comp_id, extra_data) VALUES (?, ?, ?)
2021-06-18 16:02:31,567 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ((0, 6, ''), (1, 5, ''), (2, 3, ''), (3, 0, ''), (4, 2, ''), (5, 4, ''), (6, 1, ''), (7, 10, '')  ... displaying 10 of 21 total bound parameter sets ...  (19, 18, ''), (20, 16, ''))
2021-06-18 16:02:31,569 INFO sqlalchemy.engine.Engine COMMIT
2021-06-18 16:02:31,574 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-18 16:02:31,575 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-18 16:02:31,576 INFO sqlalchemy.engine.Engine [cached since 0.2372s ago] ()


In [23]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

2021-06-18 16:02:31,584 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  3


In [24]:
p.boreholes

2021-06-18 16:02:31,593 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-18 16:02:31,594 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-18 16:02:31,595 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ('205',)
2021-06-18 16:02:31,597 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-18 16:02:31,598 INFO sqlalchemy.engine.Engine [cached since 0.217s ago] ('205',)
2021-06-18 16:02:31,600 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "B

[<core.orm.BoreholeOrm>(Name=205, Length=4.8, Diameter=0.075, Intervals=7),
 <core.orm.BoreholeOrm>(Name=207, Length=4.8, Diameter=0.075, Intervals=7),
 <core.orm.BoreholeOrm>(Name=214, Length=4.8, Diameter=0.075, Intervals=7)]

In [25]:
p.refresh(verbose=True)

2021-06-18 16:02:31,617 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-18 16:02:31,618 INFO sqlalchemy.engine.Engine [cached since 0.2793s ago] ()
Decor({'_colour': '#ffffe9', 'width': None, 'component': Component({'lithology': 'matériau(?:x)? meuble(?:s)?'}), 'hatch': None})
Decor({'_colour': '#ffffd5', 'width': None, 'component': Component({'lithology': 'alluvion(?:s)?'}), 'hatch': None})
Decor({'_colour': '#331100', 'width': None, 'component': Component({'lithology': 'boue(?:s)?'}), 'hatch': None})
Decor({'_colour': '#d3b798', 'width': None, 'component': Component({'lithology': 'remblai(?:s)?'}), 'hatch': None})
Decor({'_colour': '#f5e1bd', 'width': None, 'component': Component({'lithology': 'lœss'}), 'hatch': None})
Decor({'_colour': '#f5e1bd', 'width': None, 'component': Component({'lithology': 'loess'}), 'hatch': None})
Decor({'_colour': '#d6c59

In [26]:
session.close()

2021-06-18 16:02:31,625 INFO sqlalchemy.engine.Engine ROLLBACK


# 3D Display

## Display one object

In [27]:
bh = p.boreholes_3d
#print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

In [37]:
n = 2
df.query(f'ID=="{p.boreholes_3d[n].name}"')[['ID', 'Litho_top', 'Litho_base', 'Description']]

,ID,Litho_top,Litho_base,Description
14,214,0.0,0.6,Remblais noirs en mélange avec sable beige
15,214,0.6,1.0,Remblais noirs gravillonneux
16,214,1.0,1.2,Remblais de déchets de construction brun/beige...
17,214,1.2,2.4,Remblais avec briquaillons divers bruns/orange...
18,214,2.4,3.2,Remblais avec strate de produit pur
19,214,3.2,3.6,Remblais en mélange avec limon verdâtre - prés...
20,214,3.6,4.8,Retombées et limons verdâtre argilo-sableux - ...


In [29]:
p.boreholes_3d[n].intervals[0].components[0].lithology #components

'remblais'

In [30]:
import re
for intv in p.boreholes_3d[n].intervals:
    comp = intv.components[0]
    #print('---------------\n', p.boreholes_3d[n].components)
    if hasattr(comp, 'lithology'):
        comp_litho = comp.lithology
        for leg in p.boreholes_3d[n].legend:
            leg_litho = leg.component.lithology
            reg = re.compile("^{:s}$".format(leg_litho), flags=re.I).match(comp_litho)
            if reg:  # lithology found
                print(reg.string, '---', leg.colour)

remblais --- #d3b798
remblais --- #d3b798
sable --- #ffcb23
limon --- #ffd345


SyntaxError: invalid syntax (<ipython-input-32-caa8c61c0771>, line 1)

In [33]:
p.plot3d() #205,207,214

ViewInteractiveWidget(height=400, layout=Layout(height='auto', width='100%'), width=600)

In [ ]:
pause

### Display project boreholes

In [34]:
p.plot3d(labels_size=15, bg_color=["royalblue", "aliceblue"], window_size=(1300, 1000))

3


### Boreholes in 3D using X3D for interaction within the notebook

#---------------------------------------------